In [1]:
include("../src/jac.jl")  ## using JAC

┌ Info: Precompiling JAC [top-level]
└ @ Base loading.jl:1187
ERROR: LoadError: LoadError: syntax: incomplete: "module" at /home/fritzsch/fri/JAC/src/module-Hfs.jl:6 requires end
Stacktrace:
 [1] include at ./boot.jl:317 [inlined]
 [2] include_relative(::Module, ::String) at ./loading.jl:1038
 [3] include at ./sysimg.jl:29 [inlined]
 [4] include(::String) at /home/fritzsch/fri/JAC/src/JAC.jl:39
 [5] top-level scope at none:0
 [6] include at ./boot.jl:317 [inlined]
 [7] include_relative(::Module, ::String) at ./loading.jl:1038
 [8] include(::Module, ::String) at ./sysimg.jl:29
 [9] top-level scope at none:2
 [10] eval at ./boot.jl:319 [inlined]
 [11] eval(::Expr) at ./client.jl:399
 [12] top-level scope at ./none:3
in expression starting at /home/fritzsch/fri/JAC/src/module-Hfs.jl:2
in expression starting at /home/fritzsch/fri/JAC/src/JAC.jl:73


LoadError: Failed to precompile JAC [top-level] to /home/fritzsch/.julia/compiled/v0.7/JAC.ji.

Multiply charged ions have attracted contineous interest due to recent observations by the Solar and heliospheric Observatory (SOHO), the Far Ultraviolet Spectroscopic Explorer (FUSE) and several others. In the extreme ultra-violet (EUV) range of spectra, for instance, many of the spectroscopically observed lines belong to the iron-group elements owing to their high abundance in different objects. Until the present, however, many of these lines remained unclassified because reliable theoretical predictions are often not available that consistently include relativistic and correlation effects. 

To demonstrate the use of the JAC toolbox, we here consider the level structure of Fe^9+ ions which, in spectroscopic notation, give rise to the spectrum Fe X. Several medium-to-large scale computations have been carried out to calculate the excitation energies and transition probabilities of (and among) the levels from the
[Ne] 3s^2 3p^5  --->  3s^2 3p^4 3d + 3s 3p^6 configuration. These three configurations comprises in total
31 fine-structure levels with total angular momenta J=1/2, ..., 9/2 (cf. Dong et al., 1999). Although the 
3s^2 3p^4 3d + 3s 3p^6 levels mainly decay by electric-dipole (E1) transitions, also M1, E2 and M2 multipole transitions have been considered in the literature. Below, we show how this complete Fe X spectrum can be calculated approximately by means of a single call within the JAC toolbox.



However, before we shall perform this computation, let us first consider some technical issues of how to select
proper *physical units*, in which all the energies, rates, etc. are handled at input and output time. To see the present **settings** of this units, we can simply call:

In [2]:
JAC.display("settings")

UndefVarError: UndefVarError: JAC not defined

Of course, we could easily change any of these units by a proper call to JAC.define(), but we shall leave the present (standard) choice for the given computations. Call JAC.define() to change these settings:


In [3]:
? JAC.define

UndefVarError: UndefVarError: JAC not defined


Moreover, we can call JAC.define() to open some ASCII (.txt) file into which all the final (important) results are printed for later reference and inspection.


In [4]:
JAC.define("print summary: open", "06-output-radiative.txt")

UndefVarError: UndefVarError: JAC not defined


Next, we shall now describe the computation to be carried out. Apart from a title line (String) and nuclear model (cf. tutorial 03-setting-the-nucleus.ipynb), we here need to specify the initial- and final-state configurations that are to be included into the computation. This is done by defining an instance of an:

comp = Atomic.Computation("Energies and Einstein coefficients for the spectrum Fe X",  Nuclear.Model(26.);              
              initialConfigs = [Configuration("[Ne] 3s 3p^6"), Configuration("[Ne] 3s^2 3p^4 3d")],              
              finalConfigs   = [Configuration("[Ne] 3s^2 3p^5")],               
              process = RadiativeX, 
              processSettings = Radiative.Settings(...) ); 
              
In principle, any number of configurations can be specified as long as they are listed explicitly. Here, we have selected also the identifier **RadiativeX** to consider the computation of transition probabilities between the levels of the three configurations given above. All further details about any computation can be specified by means of some proper **settings** that are specific to each particular process, here Radiative.Settings(). To better understand to which extent we can affect and control these computations, let us loot at the definition of this data struct:


In [5]:
? Radiative.Settings

UndefVarError: UndefVarError: Radiative not defined


Of course, we can always apply the default values for any particular compuation which, for present case, are given by:

JAC.Radiative.Settings()
Radiative.Settings(with multipoles = EmMultipole[E1], use-gauges = UseGauge[JAC.UseBabushkin], calcAnisotropy = false, printBeforeComputation = false, selectLines = false, selectedLines = Tuple{Int64,Int64}[], photonEnergyShift = 0.0, mimimumPhotonEnergy = 0.0, maximumPhotonEnergy = 0.0) 


Below, we wish to consider also other multipole transitions, namely E1, M1, E2, M2, and like perform all calculations within both, the Coulomb and Babushkin gauges. In addition, we shall NOT compute the (angular) anisotropy parameters nor do we wish to select individual transitions right from the beginning. This latter option might be interesting if large sets of configurations are to be considered. Moreover, we do not neither specify any (overall) shift nor restrict the photon energies, two features that are of particular interest for large data sets


We these remarks in mind, we now specify the atomic computation in the form:


In [6]:
comp = Atomic.Computation("Energies and Einstein coefficients for the spectrum Fe X",  Nuclear.Model(26.);
              initialConfigs = [Configuration("[Ne] 3s 3p^6"), Configuration("[Ne] 3s^2 3p^4 3d")],
              finalConfigs   = [Configuration("[Ne] 3s^2 3p^5")], 
              process = RadiativeX, 
              processSettings = Radiative.Settings([E1, M1, E2, M2], [JAC.UseCoulomb, JAC.UseBabushkin], 
                        false, true, false, Tuple{Int64,Int64}[], 0., 0., 10000. ) ); 

UndefVarError: UndefVarError: Nuclear not defined


All what is needed now is to **perform** this computation by the call:


In [7]:
perform(comp)

UndefVarError: UndefVarError: perform not defined


... and which shows all the intermediate output of the computations. First, all the initial-state levels are calculated *self-consistently* due to the default values for such SCF and configuration-interaction computations, and similarly for all final-state levels. These computations include the definition of the CSF basis, the generation of *start* orbitals as well as the computation of the angular coefficients, the SCF interation and up to the set-up and diagonalization of the corresponding Hamiltonian matrix. Obviously, this gives rise to a full representation of all atomic levels of interest.

These levels are then utilized to compute the (multipole) transitions amplitudes and rates. All final results are also tabulated in a neat format. However, since many of the internal tables of the JAC toolbox are *wider* than the standard widths of these notebooks, we shall first **close** our .txt summary file:


In [8]:
JAC.define("print summary: close", "")

UndefVarError: UndefVarError: JAC not defined


This summary file **06-output-radiative.txt** can be found in the currnt directory. As seen from these tables, the transition energies agree within 3..4 digits with a similar computation by other atomic structure codes, such as GRASP. Apart from the level splitting among the low-lying 3s 3p^6, 3s^2 3p^4 3d  J=1/2 levels, further comparision can be made also with the evaluated data from the NIST Spectroscopic Database, for instance, for the excitation energies from the 3s^2 3p^5  J = 3/2 ground state.